# Продвинутый Python, семинар 5

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Бузаев Федор, Коган Александра, Дешеулин Олег

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Что сегодня?

Ура мы заканчиваем исследование наших данных по TFT

Но вначале надо доисследовать данные, разобраться с Plotly и Dash и показать всю его мощь

Поехали!

![](https://pbs.twimg.com/media/D6MBvrkUIAEQlIi.jpg)

## Plotly

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'pandas'

In [ ]:
!wget https://github.com/Palladain/Deep_Python/raw/main/Seminars/Seminar_2/archive.zip

--2022-09-28 19:38:48--  https://github.com/Palladain/Deep_Python/raw/main/Seminars/Seminar_2/archive.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Palladain/Deep_Python/main/Seminars/Seminar_2/archive.zip [following]
--2022-09-28 19:38:48--  https://raw.githubusercontent.com/Palladain/Deep_Python/main/Seminars/Seminar_2/archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694969 (679K) [application/zip]
Saving to: ‘archive.zip’

archive.zip         100%[===================>] 678.68K  --.-KB/s    in 0.1s    

2022-09-28 19:38:50 (6.45 MB/s) - ‘archive.zip’ saved [694969/694969]



In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: items.json              
  inflating: tft_3.5_last_500_gm_match_info.csv  
  inflating: tft_3.5_last_500_gm_unit_info.csv  
  inflating: traits.json             


In [ ]:
heroes = pd.read_csv('tft_3.5_last_500_gm_unit_info.csv')

matches = pd.read_csv("tft_3.5_last_500_gm_match_info.csv")
matches["winner"] = (matches.placement <= 4).astype("int")
matches.game_datetime = matches.game_datetime.apply(lambda x: parse(x))
matches['hour'] = matches.game_datetime.apply(lambda x: x.hour)
matches['day'] = matches.game_datetime.apply(lambda x: x.day)
matches['galaxy_numb'] = pd.factorize(matches.galaxy)[0]

Итак, с чего начнем?

Вспоминаем, как рисовали 24 графика по поводу каждой особенности. Выглядит классно и интересно, но смотреть столько графиков одновременно - немного жутко.

Вот хочется как на дашборде: берешь такой, протыкиваешь фильтры и пожалуйста, одна картинка

Но теперь мы знаем Plotly и с его помощью это и сделаем и покажем его интерактивность

### Задание 1

Сделайте график, который показывает распределение мест в зависимости от

Первое, что нам нужно сделать - нарисовать для каждой колонки свой собственный график, чтобы в зависимости от фильтра показывать нужное

In [ ]:
matches.columns

Index(['match_id', 'puuid', 'challenger', 'galaxy', 'placement',
       'game_datetime', 'Astro', 'Chrono', 'DarkStar', 'Set3_Celestial',
       'Set3_Mystic', 'Sniper', 'Vanguard', 'Cybernetic', 'Infiltrator',
       'ManaReaver', 'Battlecast', 'Blaster', 'Set3_Brawler', 'Rebel',
       'Starship', 'Demolitionist', 'Mercenary', 'Set3_Blademaster', 'Paragon',
       'Protector', 'Set3_Sorcerer', 'StarGuardian', 'MechPilot',
       'SpacePirate', 'winner', 'hour', 'day'],
      dtype='object')

Как это можно сделать?

In [ ]:
cols = ['Astro', 'Chrono', 'DarkStar', 'Set3_Celestial',
       'Set3_Mystic', 'Sniper', 'Vanguard', 'Cybernetic', 'Infiltrator',
       'ManaReaver', 'Battlecast', 'Blaster', 'Set3_Brawler', 'Rebel',
       'Starship', 'Demolitionist', 'Mercenary', 'Set3_Blademaster', 'Paragon',
       'Protector', 'Set3_Sorcerer', 'StarGuardian', 'MechPilot',
       'SpacePirate']

fig = go.Figure()

#нарисуем все распределения
for col in cols:
    fig.add_trace(go.Box(x=matches[col], y=matches["placement"], visible=True, opacity=0.6))

# Наше меню для отображения
updatemenu= [dict()]

# Кнопки, на которые можно нажимать
buttons=[]

for i, col in enumerate(cols):
    visibles = [False] * len(cols)
    visibles[i] = True
    buttons.append(dict(method='restyle',
                        label=col,
                        args=["visible", visibles]
                        ))

updatemenu[0]['buttons']=buttons
updatemenu[0]['direction']='down'
updatemenu[0]['showactive']=True

fig.update_layout(showlegend=False, updatemenus = updatemenu)
go.layout.Updatemenu(x=0, y=1)
fig.update_layout(
    title = {
        "text": "Placement distribution",
        "x": 0.5
    })

fig.show()

### Задание 2


Сделайте график со слайдером c началом игры и их числу

In [ ]:
fig = go.Figure()

days = matches.groupby("game_datetime").agg({"match_id": "nunique"}).reset_index()

fig.add_trace(go.Scatter(x=days.game_datetime, y=days.match_id))

fig.update_layout(
    title = {
        "text": "Number of matches",
        "x": 0.5
    }
)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

Что-то не очень хорошо получилось, правда? (Потому что каждая игра начиналась в уникальное время)

Давайте сделаем также, но только оставив дату и час

### Задание 3

Сделайте график со слайдером c началом игры (в виде дата + час) и их числу

In [ ]:
def strip_date(x):
    x = x.replace(minute=0)
    x = x.replace(second=0)
    return x

matches["gametime"] = matches.game_datetime.apply(lambda x: strip_date(x))

fig = go.Figure()

days = matches.groupby("gametime").agg({"match_id": "nunique"}).reset_index()

fig.add_trace(go.Scatter(x=days.gametime, y=days.match_id))

fig.update_layout(
    title = {
        "text": "Number of matches",
        "x": 0.5
    }
)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

Что еще можем посмотреть интересного? Можем посмотреть, например, на использование чемпионов во времени!

### Задание 4

Отобразите динамику использования чемпионов и их винрейта во времени

In [ ]:
res = pd.merge(matches[['match_id', 'puuid', 'gametime', "winner"]], heroes[["match_id", "puuid", "champion", "traits_1"]], on=['match_id', 'puuid'], how='inner')
res = res.groupby(["champion", "gametime", "traits_1"]).agg({"puuid": "nunique", "winner": "sum"}).reset_index()
res["gametime"] = res.gametime.apply(lambda x: str(x))
res["perc"] = res.winner / res.puuid

In [ ]:
px.scatter(res, x="puuid", y="perc", animation_frame="gametime", animation_group="champion",
           size="winner", color="traits_1", hover_name="champion", range_x=[-1,50], range_y=[-1,2])

К сожалению в нашем датасете нет никаких координат, поэтому для карт давайте вернемся к нашим картам из лекции

In [ ]:
cities = pd.read_csv('https://raw.githubusercontent.com/hflabs/city/master/city.csv')
cities.head()

address  postal_code country federal_district region_type  \
0   Респ Адыгея, г Адыгейск     385200.0  Россия            Южный        Респ   
1                  г Майкоп     385000.0  Россия            Южный        Респ   
2           г Горно-Алтайск     649000.0  Россия        Сибирский        Респ   
3  Алтайский край, г Алейск     658125.0  Россия        Сибирский        край   
4                 г Барнаул     656000.0  Россия        Сибирский        край   

      region area_type area city_type           city  ... fias_level  \
0     Адыгея       NaN  NaN         г       Адыгейск  ...          4   
1     Адыгея       NaN  NaN         г         Майкоп  ...          4   
2      Алтай       NaN  NaN         г  Горно-Алтайск  ...          4   
3  Алтайский       NaN  NaN         г         Алейск  ...          4   
4  Алтайский       NaN  NaN         г        Барнаул  ...          4   

  capital_marker        okato        oktmo  tax_office  timezone    geo_lat  \
0              0  79403000000  79703000001         107     UTC+3  44.878414   
1              2  79401000000  79701000001         105     UTC+3  44.609827   
2              2  84401000000  84701000001         400     UTC+7  51.958103   
3              0   1403000000   1703000001        2201     UTC+7  52.492251   
4              2   1401000000   1701000001        2200     UTC+7  53.347997   

     geo_lon  population foundation_year  
0  39.190289       12689            1969  
1  40.100661      144055            1857  
2  85.960324       62861            1830  
3  82.779361       28528            1913  
4  83.779806      635585            1730  

[5 rows x 24 columns]

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'], lon=cities['geo_lon'], text=cities['city']))
capital = cities[cities['region']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

Пока что выглядит скудно. Давайте немного добавим ей информативности!

### Задание 5

Посмотрите на данные по году основания города и населению. Нанесите на карту с помощью цвета информацию о годе основания и с помощью размера население

In [ ]:
cities.foundation_year.unique()

In [ ]:
cities.population.unique()

array([ 12689, 144055,  62861, ...,  41001,  34505, 591486])

In [ ]:
def to_int_size(value):
    try:
        return np.log10(int(value))
    except:
        return np.log10(int(value.split('[')[0]))

def to_int_year(value):
    try:
        return int(value)
    except:
        return None

cities['foundation_year'] = cities['foundation_year'].map(to_int_year)

fig = go.Figure(go.Scattermapbox(lat=cities['geo_lat'],
                                 lon=cities['geo_lon'],
                                 text=cities['city'],
                                 marker=dict(colorbar=dict(title="Год основания"),
                                             color=cities['foundation_year'],
                                             size=cities['population'].map(to_int_size))))
capital = cities[cities['region']=='Москва']
map_center = go.layout.mapbox.Center(lat=capital['geo_lat'].values[0], lon=capital['geo_lon'].values[0])
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2))
fig.show()


Красота! Города есть, остались дороги!

### Задание 6

Используя данные по путешествию на поезде Москва-Владивосток, изобразите жд линию на карте с учетом числа дней поездки

In [ ]:
train_russia = pd.read_csv('https://gist.githubusercontent.com/lexnekr/2da07b5fc12b5be24068e4d68ed47ca5/raw/d6256765a3223282fbfec7e0b040cbfb21593fff/train_russia.scv')
train_russia.head(5)

day number             city    geo_lat    geo_lon
0           1         г Москва  55.754047  37.620405
1           1         Владимир  56.128080  40.408438
2           1  Нижний Новгород  56.324063  44.005391
3           1          Семенов  56.788979  44.490288
4           1            Киров  58.603526  49.667930

In [ ]:
data = [go.Scattermapbox(legendgroup="group",
                         name='Города России',
                         lat=cities['geo_lat'],
                         lon=cities['geo_lon'],
                         text=cities['city'],
                         marker=dict(colorbar=dict(title="Год основания"),
                                     color=cities['foundation_year'].map(to_int_year),
                                     size=cities['population'].map(to_int_size)))]
for df_for_today in train_russia.groupby(['day number']):
    data.append(go.Scattermapbox(visible=False,
                                 name='День {}'.format(df_for_today[0]),
                                 mode = "lines",
                                 hoverinfo='skip',
                                 lat=df_for_today[1]['geo_lat'],
                                 lon=df_for_today[1]['geo_lon']))



fig = go.Figure(data)

frames=[]
for i in range(len(data)+1):
    temp_frame = go.Frame(name=str(i), data=data)

    for j in range(1, i):
        temp_frame['data'][j]['visible']=True


    frames.append(temp_frame)

steps = []
for i in range(len(data)):
    step = dict(
        label = str(i),
        method = "animate",
        args = [[str(i+1)]]
    )
    steps.append(step)

sliders = [dict(
    currentvalue = {"prefix": "День №", "font": {"size": 20}},
    len = 0.9,
    x = 0.1,
    pad = {"b": 10, "t": 50},
    steps = steps,
)]

map_center = go.layout.mapbox.Center(lat=(cities['geo_lat'].max()+cities['geo_lat'].min())/2,
                                     lon=(cities['geo_lon'].max()+cities['geo_lon'].min())/2)
fig.update_layout(
                  legend_orientation="h",
                  mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2),
                  updatemenus=[dict(direction="left",
                                    pad = {"r": 10, "t": 80},
                                    x = 0.1,
                                    xanchor = "right",
                                    y = 0,
                                    yanchor = "top",
                                    showactive=False,
                                    type="buttons",
                                    buttons=[dict(label="►", method="animate", args=[None, {"fromcurrent": True}]),
                                             dict(label="❚❚", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                                                               "mode": "immediate",
                                                                                               "transition": {"duration": 0}}])])],
                  )


fig.layout.sliders = sliders
fig.frames = frames

fig.show()

Отлично! Теперь потренируемся с Dash на немного других данных

Будем разбирать данные [голосований](https://www.kaggle.com/datasets/etsc9287/2020-general-election-polls?select=county_statistics.csv) по округам в 2016 и 2020 годах в США (попытаемся понять, кто все эти люди, которые голосовали за демократов и республиканцев)

### Грузим данные, смотрим, что есть

In [ ]:
!wget https://github.com/Palladain/Deep_Python/raw/main/Seminars/Seminar_11/county_statistics.csv

In [ ]:
import pandas as pd

df = pd.read_csv("county_statistics.csv")
df.head(5)

Unnamed: 0     county state  percentage16_Donald_Trump  \
0           0  Abbeville    SC                      0.629   
1           1     Acadia    LA                      0.773   
2           2   Accomack    VA                      0.545   
3           3        Ada    ID                      0.479   
4           4      Adair    IA                      0.653   

   percentage16_Hillary_Clinton  total_votes16  votes16_Donald_Trump  \
0                         0.346        10724.0                6742.0   
1                         0.206        27386.0               21159.0   
2                         0.428        15755.0                8582.0   
3                         0.387       195587.0               93748.0   
4                         0.300         3759.0                2456.0   

   votes16_Hillary_Clinton  percentage20_Donald_Trump  percentage20_Joe_Biden  \
0                   3712.0                      0.661                   0.330   
1                   5638.0                      0.795                   0.191   
2                   6737.0                      0.542                   0.447   
3                  75676.0                      0.504                   0.465   
4                   1127.0                      0.697                   0.286   

   ...  Walk  OtherTransp  WorkAtHome  MeanCommute  Employed  PrivateWork  \
0  ...   1.8          1.8         6.5         25.8    9505.0         78.8   
1  ...   1.6          2.2         2.5         27.6   24982.0         80.0   
2  ...   2.6          1.8         4.5         22.0   13837.0         74.6   
3  ...   1.5          2.8         6.9         20.4  214984.0         78.3   
4  ...   2.8          0.4         6.2         22.3    3680.0         73.8   

   PublicWork  SelfEmployed  FamilyWork  Unemployment  
0        13.3           7.8         0.1           9.4  
1        12.1           7.6         0.3           8.9  
2        18.1           7.1         0.2           5.4  
3        15.0           6.6         0.1           4.3  
4        15.3          10.4         0.5           3.0  

[5 rows x 51 columns]

In [ ]:
df.columns

Index(['Unnamed: 0', 'county', 'state', 'percentage16_Donald_Trump',
       'percentage16_Hillary_Clinton', 'total_votes16', 'votes16_Donald_Trump',
       'votes16_Hillary_Clinton', 'percentage20_Donald_Trump',
       'percentage20_Joe_Biden', 'total_votes20', 'votes20_Donald_Trump',
       'votes20_Joe_Biden', 'lat', 'long', 'cases', 'deaths', 'TotalPop',
       'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian',
       'Pacific', 'VotingAgeCitizen', 'Income', 'IncomeErr', 'IncomePerCap',
       'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Professional', 'Service',
       'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed',
       'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork',
       'Unemployment'],
      dtype='object')

Что у нас есть? У нас есть counties (можно считать как округа), на которые поделен каждый штат и по каждому из них у нас есть результаты голосований, а также различные социоэкономические данные и данные по COVID

Что хотелось бы базово отрисовать? Изобразить все округа и дать по каждой информацию

In [ ]:
df = df[~pd.isnull(df.lat)]
df = df[~pd.isnull(df.percentage16_Donald_Trump)]

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(data_frame=df, lat='lat', lon='long', text='county', zoom=3)
fig.update_layout(mapbox_style="open-street-map")
fig.update_traces(hoverinfo="all", hovertemplate="%{text}")

Так, что-то да сделали. Но вот что не очень: у нас county - это есть ни что иное, как округ. Он выражается не одной точкой, а полигоном. Давайте сделаем так, чтобы отображались полигоны, а не просто точки

Для этого нам нужны знания из интернетов. У полигонов по США есть идентификатор (который называется FIPS, представляет из себя код штата + код округа). Давайте быстренько спарсим:

In [ ]:
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

def replacing_words(x):
    x = x.split(",")[0]
    samples = [" County", "/town", " Parish", "/city"]
    for s in samples:
        x = x.replace(s, "")
    return x

def changing_state(x):
    x = x.split(", ")[-1]
    if x == 'District of Columbia':
        x = "DC"
    return x

df_sample["county"] = df_sample["County Name/State Abbreviation"].apply(lambda x: replacing_words(x))
df_sample["state"] = df_sample["County Name/State Abbreviation"].apply(lambda x: changing_state(x))
df_fips = pd.merge(df, df_sample[["county", "state", "FIPS"]], on=["county", "state"], how='left')

Ура, у нас есть FIPS, теперь можем легко отрисовать штаты!

In [ ]:
from urllib.request import urlopen
import json
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

fig = px.choropleth(df_fips, geojson=counties, locations='FIPS', color='White',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                            scope="usa",
                            fitbounds='locations',
                           hover_data=["county", "White"]
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

Красота!

Теперь давайте добавим какие-нибудь селекторы, например, выбор штата (чтобы отображать на карте только выбранный штат)

### Задание 7

Добавьте селектор со штатом так, чтобы при изменении на карте показывался только необходимый штат

А также добавьте селектор, который будет показывать разные параметры для просмотра (не только процент белых, но и остальные вещи, которые мы могли бы посмотреть)

Отлично, теперь мы, наверное, хотим еще добавить селектор отдельно на county, чтобы посмотреть, что там вообще происходит

### Задание 8

Придумайте не менее 4 графиков для конкретного county (что можно посмотреть отдельно по округу)

А также придумайте не менее 4 графиков для всего штата (чтобы можно было смотреть также и статистику по штату)

Прекрасно, у нас теперь есть отдельно выбор county. Но подождите... А зачем мы карту тогда рисовали? А сейчас мы уберем последний селектор и натянем все на карту, чтобы можно все делать по клику! (Понятно ли, почему не по hover делать будем?)

### Задание 9

Преобразуйте результат задания 3 так, чтобы можно было убрать последний селектор и делать все вещи по клику (соответственно, если клика нет, то информацию должен показывать по штату)

И естественно внутри Dash можно сделать странички (это же просто приложение на Flask)

### Задание 10

Сделайте вторую страничку, где можно выбрать два штата и сравнить по значениям между ними